**Welcome 
- **The chatbot should look at your question and look around the internet for some resources.**
- **Based on those resources, the chatbot should make an educated guess based on its retrieved information.**

In [108]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_nvidia import ChatNVIDIA

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

<hr><br>

## **Part 1:** Define The Planner


In [ ]:
from pydantic import BaseModel, Field
from functools import partial
from typing import List

from course_utils import SCHEMA_HINT


## Create an LLM client with the sole intention of generating a plan.

class Plan(BaseModel):
    steps: List[str]


In [123]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser

# Instantiate a PydanticOutputParser for the Plan model
plan_parser = PydanticOutputParser(pydantic_object=Plan)

# Get format instructions (schema hint) for the Plan output
schema_hint = plan_parser.get_format_instructions().replace("{", "{{").replace("}", "}}")


In [124]:
# Escape curly braces in schema_hint to prevent formatting errors
escaped_schema_hint = schema_hint.replace("{", "{{").replace("}", "}}")

# Final planning prompt with improved prompt engineering
planning_prompt = ChatPromptTemplate.from_messages([
    ("system", f"""You are a master planner system who charts out a plan for how to solve a problem.

Only respond with a valid JSON object matching the format below.
Do NOT include any commentary, explanation, or additional text outside the JSON.

{escaped_schema_hint}
"""),
    ("human", "{input}")
])


In [125]:
planning_chain = planning_prompt | llm | plan_parser

# Step 6: Run the chain
input_msgs = {"input": "Can you help me learn more about LangGraph?"}
plan = planning_chain.invoke(input_msgs)

# Step 7: Print output
print(plan.steps)

['Read LangGraph documentation to understand its purpose and functionality', 'Explore LangGraph website and download resources related to the project', 'Watch LangGraph tutorials and demonstrations on YouTube or other video sharing platforms', 'Familiarize yourself with the programming languages and technologies used in LangGraph ', 'Join online communities or forums to ask questions and interact with other LangGraph users', 'Experiment with LangGraph tools and libraries to gain hands-on experience', 'Read case studies or examples of projects that have successfully used LangGraph to solve real-world problems', 'Participate in LangGraph community events, meetups, or hackathons to network with other users and developers']


<br>

Parser

In [ ]:
def generate_thoughts(input_msgs, config=None):
    step_buffer = [""]
    for chunk in planning_chain.stream(input_msgs, config=config):
        if "steps" in chunk and chunk.get("steps"):
            if len(chunk.get("steps")) > len(step_buffer):
                yield step_buffer[-1]
                step_buffer += [""]
            dlen = len(chunk.get("steps")[-1]) - len(step_buffer[-1])
            step_buffer[-1] = chunk.get("steps")[-1]
    yield step_buffer[-1]
    print("FINISHED", flush=True)

from time import sleep

for thought in generate_thoughts(input_msgs):
    
    print("-", thought)
    


- 
FINISHED


<hr><br>

## Define The Retrieval Sub-Process Mechanism


In [127]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_core.runnables import RunnableLambda
import functools

# Step 1: Optional caching of internet search
@functools.cache
def search_internet(query: str):
    search = DuckDuckGoSearchAPIWrapper()
    results = search.run(query)
    return results

# Step 2: Create a Runnable that wraps the search logic
search_tool = RunnableLambda(lambda step: search_internet(step))

# Step 3: Batch-style research process
def research_options(steps):
    valid_steps = [step.strip() for step in steps if isinstance(step, str) and step.strip()]
    return search_tool.batch(valid_steps)


In [128]:
from langchain_nvidia import NVIDIARerank
from langchain_core.documents import Document

# Initialize Reranker
reranker = NVIDIARerank()

# Rerank retrieved search results using the original query (step)
def retrieve_via_query(context_rets, query: str, k=5):
    documents = [Document(page_content=ret) for ret in context_rets]
    ranked_docs = reranker.compress_documents(documents=documents, query=query)
    return [doc.page_content for doc in ranked_docs[:k]]


/usr/local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_common.py:176: UserWarning: An API key is required for the hosted NIM. This will become an error in the future.
  warnings.warn(


<hr><br>

## **Part 3:** Creating The Research Pipeline

Now that we have some minimum-viable semblance of a supervisor/subordinate system, let's go ahead and orchestrate them in an interesting way. Feel free to come up with your own mechanism for "reasoning" about the question and "researching" the results. If you don't see a straightforward way to make it work, a default pool of prompts is offered below (possibly the ones we used).

In [129]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """You are an agent tasked with providing an accurate, concise, and informative answer to a user’s query.
        You will be given a question, a structured plan with steps, and retrievals containing relevant context from internet searches.
        Provide a detailed yet concise answer that directly addresses the user's original question.
        Explicitly cite the most relevant sources from the provided context at the end."""
    ),
    ("human", """
Question: {question}

Structured Plan:
{steps}

Retrieved Context:
{retrievals}
""")
])

# Define reasoning pipeline
research_pipeline = agent_prompt | llm | StrOutputParser()

# Example usage
question = "Can you help me learn more about LangGraph?"

# Generate the structured plan
input_msgs = {"input": question}
sequence_of_actions = [thought for thought in generate_thoughts(input_msgs)]

# Perform internet searches for each step
search_retrievals = research_options(sequence_of_actions)

# Rerank retrievals for each step
filtered_results = [retrieve_via_query(search_retrievals, step) for step in sequence_of_actions]

# Prepare final input
final_input = {
    "question": question,
    "steps": "\n".join(sequence_of_actions),
    "retrievals": "\n\n".join("\n".join(res) for res in filtered_results)
}

# Invoke the research pipeline
answer = research_pipeline.invoke(final_input)

# Output final answer
print("\n" + "*" * 60)
print("✅ Final Answer:\n")
print(answer)



FINISHED

************************************************************
✅ Final Answer:

LangGraph is a graph-based language model that has been gaining attention in the natural language processing (NLP) community recently. Here's an overview:

LangGraph is a hierarchical graph-based language model that was introduced in a 2021 research paper by a team of researchers from Google and the MIT-IBM Watson AI Lab ( Lewis et al., 2021). The model is designed to capture the hierarchical structure of language, which includes both local and long-range dependencies between words in a sentence.

Key Features of LangGraph:

1. **Hierarchical Structure**: LangGraph represents a sentence as a graph, where each node is a word and edges between nodes capture the relationships between words. This hierarchical structure allows the model to capture both local and long-range dependencies.
2. **Graph-Based Encoder**: The LangGraph model uses a graph-based encoder to process the input sentence. This encoder 

In [130]:
question = "Can you help me learn more about LangGraph?"
input_msgs = {"messages": [("user", question)]}

# Generate structured steps
sequence_of_actions = [thought for thought in generate_thoughts({"input": question})]

# Perform retrievals and reranking for each step
search_retrievals = research_options(sequence_of_actions)
filtered_results = [retrieve_via_query(search_retrievals, step) for step in sequence_of_actions]

# Accumulate intermediate question-answer pairs progressively
progressive_reasoning = []
for action, result in zip(sequence_of_actions, filtered_results):
    progressive_reasoning.append(f"Step: {action}\nResult: {' '.join(result)}\n")

# Concatenate the reasoning into final input
input_msgs["messages"].append(("assistant", "\n".join(progressive_reasoning)))

# Now input_msgs["messages"] contains accumulated reasoning steps
print("*" * 64)
final_input = {
    "question": question,
    "steps": "\n".join(sequence_of_actions),
    "retrievals": "\n\n".join("\n".join(res) for res in filtered_results)
}

for token in research_pipeline.stream(final_input):
    if "\n" in token:
        print(flush=True)
    else:
        print(token, end="", flush=True)


FINISHED
****************************************************************
LangGraph is a graph-based language model that has been gaining attention in the natural language processing (NLP) community recently. Here's an overview
LangGraph is a hierarchical graph-based language model that was introduced in a 2021 research paper by a team of researchers from Google and the MIT-IBM Watson AI Lab ( Lewis et al., 2021). The model is designed to capture the hierarchical structure of language, which includes both local and long-range dependencies between words in a sentence
Key Features of LangGraph
1. **Hierarchical Structure**: LangGraph represents a sentence as a graph, where each node is a word and edges between nodes capture the relationships between words. This hierarchical structure allows the model to capture both local and long-range dependencies
2. **Graph-Based Encoder**: The LangGraph model uses a graph-based encoder to process the input sentence. This encoder is designed to effect

<hr><br>

## **Part 4:** Accumulating Your Reasoning Traces


In [131]:
## TODO: Aggregate 8 question-trace-answer triples. 
submission = [
    {
        "question": "Can you help me learn more about LangGraph?",
        "trace": sequence_of_actions,
        "answer": answer
    },
    # Repeat the process with other specialized questions below
    {
        "question": "Can you help me learn more about LangGraph? Specifically, can you tell me about Memory Management?",
        "trace": "...",  # replace with actual reasoning trace
        "answer": "..."  # replace with actual final answer
    },
    {
        "question": "Can you help me learn more about LangGraph? Specifically, can you tell me about Pregel?",
        "trace": "...",  
        "answer": "..." 
    },
    {
        "question": "Can you help me learn more about LangGraph? Specifically, can you tell me about subgraphs?",
        "trace": "...",
        "answer": "..."
    },
    {
        "question": "Can you help me learn more about LangGraph? Specifically, can you tell me about full-duplex communication?",
        "trace": "...",
        "answer": "..."
    },
    {
        "question": "Can you help me learn more about LangGraph? Specifically, can you tell me about productionalization?",
        "trace": "...",
        "answer": "..."
    },
    # Add two of your own specialized questions here:
    {
        "question": "Your specialized question 1?",
        "trace": "...",
        "answer": "..."
    },
    {
        "question": "Your specialized question 2?",
        "trace": "...",
        "answer": "..."
    }
]
